# Question 2:
* Which problems can be solved quickly but have not been solved?
* Can you pinpoint the reason why? 
* Note: Only for Bangkok

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

# Functions

In [13]:
import numpy as np
def makeList(x):
    if isinstance(x,list):
        return x
    elif isinstance(x,str):
        return x.split(',')
    else: return np.nan

# the_df.type = the_df.type.apply(makelist)
# the_df = the_df.explode('type')

In [212]:
import requests
def reverse_geocode(lat, lng, api_key):
    # Base URL for the API
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    # Parameters for the request
    params = {
        "latlng": f"{lat},{lng}",
        "key": api_key
    }
    
    # Make the GET request
    response = requests.get(url, params=params)
    
    # Parse the JSON response
    if response.status_code == 200:
        data = response.json()
        if data.get('results'):
            return data['results'][0]['formatted_address']
        else:
            return "No address found for the given coordinates."
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
latitude = 100.48093
longitude = 13.75001
your_api_key = "YOUR_API_KEY"
address = reverse_geocode(latitude, longitude, your_api_key)
print(f"Address: {address}")

Address: Error: 400, {
   "error_message" : "Invalid request. Invalid 'latlng' parameter.",
   "results" : [],
   "status" : "INVALID_REQUEST"
}


In [213]:
import requests
def get_location_from_coordinates(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse"
    params = {
        'lat': lat,
        'lon': lon,
        'format': 'json',
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        address = data.get("address", {})
        region = address.get("state", "Region not found")
        province = address.get("county", "Province not found")
        return region, province
    else:
        return None, None

# Example usage
latitude = 100.5018
longitude = 13.7563
region, province = get_location_from_coordinates(latitude, longitude)
print(f"Region: {region}, Province: {province}")


Region: None, Province: None


In [214]:
import geopy
from geopy.geocoders import Nominatim 
import ssl
import certifi
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import options
def get_location_with_geopy(lat, lon):
    ctx = ssl.create_default_context(cafile=certifi.where())
    geopy.geocoders.options.default_ssl_context = ctx   
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.reverse((lat, lon), language="en")
    if location and location.raw.get("address"):
        address = location.raw["address"]
        region = address.get("state", "Region not found")
        province = address.get("county", "Province not found")
        return region, province
    return None, None

# Example usage
latitude = 13.7563
longitude = 100.5018 
region, province = get_location_with_geopy(latitude, longitude)
print(f"Region: {region}, Province: {province}")

Region: Region not found, Province: Province not found


# Consider by Types

### Exploding types

In [2]:
# combined_df = pd.read_csv('Combined_Cases_And_Logs.csv')
combined_df_bkk = pd.read_csv('Combined_Cases_And_Logs_Bangkok.csv')

In [3]:
# combined_df['province'].value_counts()
combined_df_bkk['province'].value_counts()

province
กรุงเทพมหานคร    624020
Name: count, dtype: int64

In [297]:
# combined_df['province'].value_counts().reset_index().to_csv("combined_df['province'].value_counts().reset_index().csv", index=False)

In [4]:
# combined_df.shape
combined_df_bkk.shape

(624020, 17)

In [5]:
# combined_df.head()
combined_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:38:45,14.0,2024-12-11 19:09:27,2024-12-11 19:39:27,0 days 00:30:00
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 18:57:33,0.0,2024-12-11 16:36:10,2024-12-11 18:58:10,0 days 02:22:00
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,5.0,0.0,2024-12-11 16:11:09,7.0,2024-12-11 15:52:44,2024-12-11 16:11:44,0 days 00:19:00
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 16:16:37,0.0,2024-12-11 15:50:52,2024-12-11 16:16:52,0 days 00:26:00
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:15:26,2.0,2024-12-11 15:16:16,2024-12-11 19:16:16,0 days 04:00:00


In [8]:
# combined_df['type'].value_counts()
combined_df_bkk['type'].value_counts()

type
อื่นๆ                                             115756
ทางเท้า                                            93539
ผิดกฎจราจร                                         40009
จุดเสี่ยง,ความปลอดภัย                              34433
ถนน,ผิดกฎจราจร                                     29823
                                                   ...  
อื่นๆ,น้ำท่วม,คนเร่ร่อน                                1
ควันดำ,แท็กซี่/รถเมล์                                  1
ถนน ทางเท้า ที่จอดรถ,ทางเท้า,อาคารสถานที่ชำรุด         1
อื่นๆ,ควันดำ,สายสื่อสาร,ฝุ่นควัน&กลิ่น&PM2.5           1
ถนน,ความสะอาด,จุดเสี่ยง,ความปลอดภัย                    1
Name: count, Length: 982, dtype: int64

In [10]:
# combined_df[combined_df['type'] =='อื่นๆ,น้ำท่วม,คนเร่ร่อน']
combined_df_bkk[combined_df_bkk['type'] =='อื่นๆ,น้ำท่วม,คนเร่ร่อน']

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
286899,2023-NUPL4X,"อื่นๆ,น้ำท่วม,คนเร่ร่อน","เขตปทุมวัน, สำนักการระบายน้ำ กทม., เพื่อนชัชชา...",น้ำท่วมขัง,"100.54015,13.74261",151/1 ถนน ราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุง...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,finish,5.0,0.0,2023-09-26 09:17:15,0.0,2023-09-25 15:26:24,2023-09-26 09:17:24,0 days 17:51:00


In [11]:
# combined_exploded_df = combined_df.copy()
combined_exploded_df_bkk = combined_df_bkk.copy()

In [14]:
# combined_exploded_df['type'] = combined_exploded_df['type'].apply(makeList)
combined_exploded_df_bkk['type'] = combined_exploded_df_bkk['type'].apply(makeList)

In [16]:
# combined_exploded_df['type'].value_counts()
combined_exploded_df_bkk['type'].value_counts()

type
[อื่นๆ]                                               115756
[ทางเท้า]                                              93539
[ผิดกฎจราจร]                                           40009
[จุดเสี่ยง, ความปลอดภัย]                               34433
[ถนน, ผิดกฎจราจร]                                      29823
                                                       ...  
[อื่นๆ, น้ำท่วม, คนเร่ร่อน]                                1
[ควันดำ, แท็กซี่/รถเมล์]                                   1
[ถนน ทางเท้า ที่จอดรถ, ทางเท้า, อาคารสถานที่ชำรุด]         1
[อื่นๆ, ควันดำ, สายสื่อสาร, ฝุ่นควัน&กลิ่น&PM2.5]          1
[ถนน, ความสะอาด, จุดเสี่ยง, ความปลอดภัย]                   1
Name: count, Length: 982, dtype: int64

In [17]:
# combined_exploded_df[combined_exploded_df['ticket_id'] == '2023-NUPL4X']
combined_exploded_df_bkk[combined_exploded_df_bkk['ticket_id'] == '2023-NUPL4X']

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
286899,2023-NUPL4X,"[อื่นๆ, น้ำท่วม, คนเร่ร่อน]","เขตปทุมวัน, สำนักการระบายน้ำ กทม., เพื่อนชัชชา...",น้ำท่วมขัง,"100.54015,13.74261",151/1 ถนน ราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุง...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,finish,5.0,0.0,2023-09-26 09:17:15,0.0,2023-09-25 15:26:24,2023-09-26 09:17:24,0 days 17:51:00


In [18]:
# combined_exploded_df = combined_exploded_df.explode('type') # specify the column to explode
# This means there will be multiple rows with same ticket_id for the ones with multiple types.

combined_exploded_df_bkk = combined_exploded_df_bkk.explode('type') # specify the column to explode
# This means there will be multiple rows with same ticket_id for the ones with multiple types.

In [19]:
# combined_exploded_df.shape
combined_exploded_df_bkk.shape

(896289, 17)

In [20]:
# combined_exploded_df.head()
combined_exploded_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:38:45,14.0,2024-12-11 19:09:27,2024-12-11 19:39:27,0 days 00:30:00
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 18:57:33,0.0,2024-12-11 16:36:10,2024-12-11 18:58:10,0 days 02:22:00
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,5.0,0.0,2024-12-11 16:11:09,7.0,2024-12-11 15:52:44,2024-12-11 16:11:44,0 days 00:19:00
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 16:16:37,0.0,2024-12-11 15:50:52,2024-12-11 16:16:52,0 days 00:26:00
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:15:26,2.0,2024-12-11 15:16:16,2024-12-11 19:16:16,0 days 04:00:00


In [21]:
# combined_exploded_df[combined_exploded_df['ticket_id'] == '2023-NUPL4X']
combined_exploded_df_bkk[combined_exploded_df_bkk['ticket_id'] == '2023-NUPL4X']

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
286899,2023-NUPL4X,อื่นๆ,"เขตปทุมวัน, สำนักการระบายน้ำ กทม., เพื่อนชัชชา...",น้ำท่วมขัง,"100.54015,13.74261",151/1 ถนน ราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุง...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,finish,5.0,0.0,2023-09-26 09:17:15,0.0,2023-09-25 15:26:24,2023-09-26 09:17:24,0 days 17:51:00
286899,2023-NUPL4X,น้ำท่วม,"เขตปทุมวัน, สำนักการระบายน้ำ กทม., เพื่อนชัชชา...",น้ำท่วมขัง,"100.54015,13.74261",151/1 ถนน ราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุง...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,finish,5.0,0.0,2023-09-26 09:17:15,0.0,2023-09-25 15:26:24,2023-09-26 09:17:24,0 days 17:51:00
286899,2023-NUPL4X,คนเร่ร่อน,"เขตปทุมวัน, สำนักการระบายน้ำ กทม., เพื่อนชัชชา...",น้ำท่วมขัง,"100.54015,13.74261",151/1 ถนน ราชดำริ แขวง ลุมพินี เขตปทุมวัน กรุง...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,finish,5.0,0.0,2023-09-26 09:17:15,0.0,2023-09-25 15:26:24,2023-09-26 09:17:24,0 days 17:51:00


In [22]:
# combined_exploded_df.to_csv('Combined_Cases_And_Logs_Exploded.csv', index=False)
combined_exploded_df_bkk.to_csv('Combined_Cases_And_Logs_Exploded_Bangkok.csv', index=False)

### Finding average duration by type

In [23]:
# combined_exploded_df = pd.read_csv('Combined_Cases_And_Logs_Exploded.csv')
combined_exploded_df_bkk = pd.read_csv('Combined_Cases_And_Logs_Exploded_Bangkok.csv')

In [24]:
# combined_exploded_df.shape
combined_exploded_df_bkk.shape

(896289, 17)

In [25]:
# combined_exploded_df.isnull().sum()
combined_exploded_df_bkk.isnull().sum()

ticket_id                   0
type                        0
organization_action         0
comment                     0
coords                      0
address                     0
subdistrict                 0
district                    0
province                    0
state                       0
star                   522692
count_reopen                0
last_activity               0
view_count                  0
timestamp_reported          0
timestamp_finish            0
duration                    0
dtype: int64

In [26]:
# combined_exploded_df.head()
combined_exploded_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:38:45,14.0,2024-12-11 19:09:27,2024-12-11 19:39:27,0 days 00:30:00
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 18:57:33,0.0,2024-12-11 16:36:10,2024-12-11 18:58:10,0 days 02:22:00
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,5.0,0.0,2024-12-11 16:11:09,7.0,2024-12-11 15:52:44,2024-12-11 16:11:44,0 days 00:19:00
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 16:16:37,0.0,2024-12-11 15:50:52,2024-12-11 16:16:52,0 days 00:26:00
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:15:26,2.0,2024-12-11 15:16:16,2024-12-11 19:16:16,0 days 04:00:00


#### Creating duration_df

In [27]:
# combined_exploded_df['duration'] = pd.to_timedelta(combined_exploded_df['duration'])
combined_exploded_df_bkk['duration'] = pd.to_timedelta(combined_exploded_df_bkk['duration'])

In [28]:
# combined_exploded_df['duration_int64'] = combined_exploded_df['duration'].astype(np.int64)
combined_exploded_df_bkk['duration_int64'] = combined_exploded_df_bkk['duration'].astype(np.int64)

In [29]:
# combined_exploded_df['duration_int64'].head()
combined_exploded_df_bkk['duration_int64'].head()

0     1800000000000
1     8520000000000
2     1140000000000
3     1560000000000
4    14400000000000
Name: duration_int64, dtype: int64

In [30]:
# combined_exploded_df.groupby('type')['duration_int64'].mean()
combined_exploded_df_bkk.groupby('type')['duration_int64'].mean()

type
BFMCC ขออนุญาตถ่ายทำภาพยนตร์       1.368054e+15
Public Event ขออนุญาตจัดกิจกรรม    3.981000e+14
กรุงเทพโปร่งใส                     5.751300e+15
ขยะ                                1.626255e+15
ขอความช่วยเหลือ                    3.062933e+15
                                       ...     
แท็กซี่/รถเมล์                     2.688940e+15
โรงเรียน                           2.122947e+15
ไฟป่า                              4.017793e+15
ไฟฟ้า                              1.176281e+16
ไฟฟ้า/แสงสว่าง                     2.088390e+16
Name: duration_int64, Length: 70, dtype: float64

In [35]:
# duration_df = combined_exploded_df.groupby('type')['duration_int64'].mean().reset_index()
duration_df_bkk = combined_exploded_df_bkk.groupby('type')['duration_int64'].mean().reset_index()

In [36]:
# duration_df.rename(columns={'duration_int64':'duration_mean_int64'}, inplace=True)
duration_df_bkk.rename(columns={'duration_int64':'duration_mean_int64'}, inplace=True)

In [37]:
# duration_df.head()
duration_df_bkk.head()

,type,duration_mean_int64
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,1.368054e+15
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14
2,กรุงเทพโปร่งใส,5.751300e+15
3,ขยะ,1.626255e+15
4,ขอความช่วยเหลือ,3.062933e+15


In [38]:
# duration_df['duration_mean_int64'].astype('timedelta64[m]')
duration_df_bkk['duration_mean_int64'].astype('timedelta64[m]')

0      950037500000 days 00:00:00
1      276458333333 days 08:00:00
2     3993958452122 days 08:44:00
3     1129343932715 days 07:41:00
4     2127036929777 days 07:10:00
                 ...             
65    1867319312652 days 01:38:00
66    1474268778610 days 01:41:00
67    2790133865248 days 05:27:00
68    8168617985106 days 00:46:00
69   14502708333333 days 08:00:00
Name: duration_mean_int64, Length: 70, dtype: timedelta64[s]

In [39]:
# duration_df['duration_mean_timedelta64[ns]'] = duration_df['duration_mean_int64'].astype('timedelta64[ns]')
duration_df_bkk['duration_mean_timedelta64[ns]'] = duration_df_bkk['duration_mean_int64'].astype('timedelta64[ns]')

In [40]:
# duration_df.head()
duration_df_bkk.head()

,type,duration_mean_int64,duration_mean_timedelta64[ns]
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,1.368054e+15,15 days 20:00:54
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14,4 days 14:35:00
2,กรุงเทพโปร่งใส,5.751300e+15,66 days 13:35:00.171056204
3,ขยะ,1.626255e+15,18 days 19:44:15.263110061
4,ขอความช่วยเหลือ,3.062933e+15,35 days 10:48:53.178879310


In [41]:
# combined_exploded_df['duration_int64_logscale'] = combined_exploded_df['duration_int64'].apply(np.log)
combined_exploded_df_bkk['duration_int64_logscale'] = combined_exploded_df_bkk['duration_int64'].apply(np.log)

In [42]:
# combined_exploded_df.head()
combined_exploded_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,star,count_reopen,last_activity,view_count,timestamp_reported,timestamp_finish,duration,duration_int64,duration_int64_logscale
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:38:45,14.0,2024-12-11 19:09:27,2024-12-11 19:39:27,0 days 00:30:00,1800000000000,28.218808
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 18:57:33,0.0,2024-12-11 16:36:10,2024-12-11 18:58:10,0 days 02:22:00,8520000000000,29.773437
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,5.0,0.0,2024-12-11 16:11:09,7.0,2024-12-11 15:52:44,2024-12-11 16:11:44,0 days 00:19:00,1140000000000,27.762049
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 16:16:37,0.0,2024-12-11 15:50:52,2024-12-11 16:16:52,0 days 00:26:00,1560000000000,28.075707
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,NaN,0.0,2024-12-11 19:15:26,2.0,2024-12-11 15:16:16,2024-12-11 19:16:16,0 days 04:00:00,14400000000000,30.298249


In [43]:
# temp_logscale_mean = combined_exploded_df.groupby('type')['duration_int64_logscale'].mean().reset_index()
temp_logscale_mean = combined_exploded_df_bkk.groupby('type')['duration_int64_logscale'].mean().reset_index()

In [44]:
temp_logscale_mean.rename(columns={'duration_int64_logscale':'duration_mean_int64_logscale'}, inplace=True)

In [45]:
temp_logscale_mean

,type,duration_mean_int64_logscale
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,33.975829
1,Public Event ขออนุญาตจัดกิจกรรม,33.617724
2,กรุงเทพโปร่งใส,34.668155
3,ขยะ,33.193146
4,ขอความช่วยเหลือ,33.863457
...,...,...
65,แท็กซี่/รถเมล์,33.681495
66,โรงเรียน,33.871308
67,ไฟป่า,33.506352
68,ไฟฟ้า,35.890449


In [46]:
temp_logscale_mean['duration_mean_int64_logscale'] = temp_logscale_mean['duration_mean_int64_logscale'].apply(np.exp)

In [47]:
temp_logscale_mean['duration_mean_timedelta64[ns]_logscale'] = temp_logscale_mean['duration_mean_int64_logscale'].astype('timedelta64[ns]')

In [48]:
temp_logscale_mean

,type,duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,5.695280e+14,6 days 14:12:07.990968908
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14,4 days 14:34:59.999999999
2,กรุงเทพโปร่งใส,1.138121e+15,13 days 04:08:40.878509701
3,ขยะ,2.603756e+14,3 days 00:19:35.600019465
4,ขอความช่วยเหลือ,5.089936e+14,5 days 21:23:13.575378059
...,...,...,...
65,แท็กซี่/รถเมล์,4.243139e+14,4 days 21:51:53.876192293
66,โรงเรียน,5.130055e+14,5 days 22:30:05.533442412
67,ไฟป่า,3.561425e+14,4 days 02:55:42.512087076
68,ไฟฟ้า,3.863885e+15,44 days 17:18:04.565345982


In [50]:
# duration_df = pd.merge(left=duration_df, right=temp_logscale_mean, how='left', on='type')
duration_df_bkk = pd.merge(left=duration_df_bkk, right=temp_logscale_mean, how='left', on='type')

In [51]:
# duration_df.head()
duration_df_bkk.head()

,type,duration_mean_int64,duration_mean_timedelta64[ns],duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,1.368054e+15,15 days 20:00:54,5.695280e+14,6 days 14:12:07.990968908
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14,4 days 14:35:00,3.981000e+14,4 days 14:34:59.999999999
2,กรุงเทพโปร่งใส,5.751300e+15,66 days 13:35:00.171056204,1.138121e+15,13 days 04:08:40.878509701
3,ขยะ,1.626255e+15,18 days 19:44:15.263110061,2.603756e+14,3 days 00:19:35.600019465
4,ขอความช่วยเหลือ,3.062933e+15,35 days 10:48:53.178879310,5.089936e+14,5 days 21:23:13.575378059


In [52]:
# temp_median = combined_exploded_df.groupby('type')['duration_int64'].median().reset_index()
temp_median = combined_exploded_df_bkk.groupby('type')['duration_int64'].median().reset_index()

In [53]:
temp_median.head()

,type,duration_int64
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,7.518300e+14
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14
2,กรุงเทพโปร่งใส,1.102350e+15
3,ขยะ,2.247600e+14
4,ขอความช่วยเหลือ,4.572000e+14


In [54]:
temp_median.rename(columns={'duration_int64':'duration_median_int64'}, inplace=True)

In [56]:
# duration_df = pd.merge(left=duration_df, right=temp_median, how='left', on='type')
duration_df_bkk = pd.merge(left=duration_df_bkk, right=temp_median, how='left', on='type')

In [57]:
# duration_df['duration_median_timedelta64[ns]'] = duration_df['duration_median_int64'].astype('timedelta64[ns]')
duration_df_bkk['duration_median_timedelta64[ns]'] = duration_df_bkk['duration_median_int64'].astype('timedelta64[ns]')

In [58]:
# duration_df.head()
duration_df_bkk.head()

,type,duration_mean_int64,duration_mean_timedelta64[ns],duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale,duration_median_int64,duration_median_timedelta64[ns]
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,1.368054e+15,15 days 20:00:54,5.695280e+14,6 days 14:12:07.990968908,7.518300e+14,8 days 16:50:30
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14,4 days 14:35:00,3.981000e+14,4 days 14:34:59.999999999,3.981000e+14,4 days 14:35:00
2,กรุงเทพโปร่งใส,5.751300e+15,66 days 13:35:00.171056204,1.138121e+15,13 days 04:08:40.878509701,1.102350e+15,12 days 18:12:30
3,ขยะ,1.626255e+15,18 days 19:44:15.263110061,2.603756e+14,3 days 00:19:35.600019465,2.247600e+14,2 days 14:26:00
4,ขอความช่วยเหลือ,3.062933e+15,35 days 10:48:53.178879310,5.089936e+14,5 days 21:23:13.575378059,4.572000e+14,5 days 07:00:00


In [59]:
# temp_mode = combined_exploded_df.groupby('type')['duration_int64'].agg(
#     lambda x: x.mode().iloc[0] if not x.mode().empty else None
# ).reset_index()
temp_mode = combined_exploded_df_bkk.groupby('type')['duration_int64'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()

In [60]:
temp_mode['duration_int64'].isnull().sum()

np.int64(0)

In [61]:
temp_mode.head()

,type,duration_int64
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,10140000000000
1,Public Event ขออนุญาตจัดกิจกรรม,398100000000000
2,กรุงเทพโปร่งใส,89160000000000
3,ขยะ,10260000000000
4,ขอความช่วยเหลือ,4710780000000000


In [62]:
temp_mode.rename(columns={'duration_int64':'duration_mode_int64'}, inplace=True)

In [63]:
temp_mode['duration_mode_timedelta64[ns]'] = temp_mode['duration_mode_int64'].astype('timedelta64[ns]')

In [64]:
temp_mode.head()

,type,duration_mode_int64,duration_mode_timedelta64[ns]
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,10140000000000,0 days 02:49:00
1,Public Event ขออนุญาตจัดกิจกรรม,398100000000000,4 days 14:35:00
2,กรุงเทพโปร่งใส,89160000000000,1 days 00:46:00
3,ขยะ,10260000000000,0 days 02:51:00
4,ขอความช่วยเหลือ,4710780000000000,54 days 12:33:00


In [65]:
# duration_df = pd.merge(left=duration_df, right=temp_mode, how='left', on='type')
duration_df_bkk = pd.merge(left=duration_df_bkk, right=temp_mode, how='left', on='type')

In [66]:
# duration_df.head()
duration_df_bkk.head()

,type,duration_mean_int64,duration_mean_timedelta64[ns],duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale,duration_median_int64,duration_median_timedelta64[ns],duration_mode_int64,duration_mode_timedelta64[ns]
0,BFMCC ขออนุญาตถ่ายทำภาพยนตร์,1.368054e+15,15 days 20:00:54,5.695280e+14,6 days 14:12:07.990968908,7.518300e+14,8 days 16:50:30,10140000000000,0 days 02:49:00
1,Public Event ขออนุญาตจัดกิจกรรม,3.981000e+14,4 days 14:35:00,3.981000e+14,4 days 14:34:59.999999999,3.981000e+14,4 days 14:35:00,398100000000000,4 days 14:35:00
2,กรุงเทพโปร่งใส,5.751300e+15,66 days 13:35:00.171056204,1.138121e+15,13 days 04:08:40.878509701,1.102350e+15,12 days 18:12:30,89160000000000,1 days 00:46:00
3,ขยะ,1.626255e+15,18 days 19:44:15.263110061,2.603756e+14,3 days 00:19:35.600019465,2.247600e+14,2 days 14:26:00,10260000000000,0 days 02:51:00
4,ขอความช่วยเหลือ,3.062933e+15,35 days 10:48:53.178879310,5.089936e+14,5 days 21:23:13.575378059,4.572000e+14,5 days 07:00:00,4710780000000000,54 days 12:33:00


In [67]:
# combined_exploded_averages_df = pd.merge(left=combined_exploded_df, right=duration_df, how='left', on='type')
combined_exploded_averages_df_bkk = pd.merge(left=combined_exploded_df_bkk, right=duration_df_bkk, how='left', on='type')

In [68]:
# combined_exploded_averages_df.head()
combined_exploded_averages_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,...,duration_int64,duration_int64_logscale,duration_mean_int64,duration_mean_timedelta64[ns],duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale,duration_median_int64,duration_median_timedelta64[ns],duration_mode_int64,duration_mode_timedelta64[ns]
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,...,1800000000000,28.218808,2.688940e+15,31 days 02:55:39.810218978,4.243139e+14,4 days 21:51:53.876192293,3.985800e+14,4 days 14:43:00,3780000000000,0 days 01:03:00
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,...,8520000000000,29.773437,1.348728e+15,15 days 14:38:48.329718004,2.344951e+14,2 days 17:08:15.071347040,1.927800e+14,2 days 05:33:00,13800000000000,0 days 03:50:00
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,...,1140000000000,27.762049,5.547520e+15,64 days 04:58:39.896619578,7.256029e+14,8 days 09:33:22.914968887,6.078600e+14,7 days 00:51:00,5520000000000,0 days 01:32:00
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,...,1560000000000,28.075707,2.732807e+15,31 days 15:06:46.610110803,6.511666e+14,7 days 12:52:46.554736235,5.802300e+14,6 days 17:10:30,62760000000000,0 days 17:26:00
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,...,14400000000000,30.298249,8.940264e+14,10 days 08:20:26.423278997,2.489583e+14,2 days 21:09:18.329546155,2.487000e+14,2 days 21:05:00,94320000000000,1 days 02:12:00


In [69]:
# combined_exploded_averages_df.to_csv('Combined_Cases_And_Logs_Exploded_Averages.csv', index=False)
combined_exploded_averages_df_bkk.to_csv('Combined_Cases_And_Logs_Exploded_Averages_Bangkok.csv', index=False)

Now, we have a DataFrame with all the necessary central tendency measures for each type.

### Plot duration by type

#### Adjustments to DataFrame

In [70]:
# combined_exploded_averages_df = pd.read_csv('Combined_Cases_And_Logs_Exploded_Averages.csv')
combined_exploded_averages_df_bkk = pd.read_csv('Combined_Cases_And_Logs_Exploded_Averages_Bangkok.csv')

In [72]:
# combined_exploded_averages_df.head()
combined_exploded_averages_df_bkk.head()

,ticket_id,type,organization_action,comment,coords,address,subdistrict,district,province,state,...,duration_int64,duration_int64_logscale,duration_mean_int64,duration_mean_timedelta64[ns],duration_mean_int64_logscale,duration_mean_timedelta64[ns]_logscale,duration_median_int64,duration_median_timedelta64[ns],duration_mode_int64,duration_mode_timedelta64[ns]
0,2024-ANZFC6,แท็กซี่/รถเมล์,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",รถติดหนักมากๆ นั่งอยู่บนรถเมล์สาย 48 สุขภาพจิต...,"100.57357,13.72769",748 ถ. สุขุมวิท แขวงคลองตัน เขตคลองเตย กรุงเทพ...,คลองตัน,คลองเตย,กรุงเทพมหานคร,finish,...,1800000000000,28.218808,2.688940e+15,31 days 02:55:39.810218978,4.243139e+14,4 days 21:51:53.876192293,3.985800e+14,4 days 14:43:00,3780000000000,0 days 01:03:00
1,2024-ZCBDC7,ความสะอาด,"ฝ่ายรักษาความสะอาดฯ เขตสัมพันธวงศ์, เขตสัมพันธ...",เรื่องขยะที่มาแยกขยะที่ถนนเยาวราช ก่อนถึงปากซอ...,"100.5059,13.74345",222 ถ. เยาวราช แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ ก...,สัมพันธวงศ์,สัมพันธวงศ์,กรุงเทพมหานคร,finish,...,8520000000000,29.773437,1.348728e+15,15 days 14:38:48.329718004,2.344951e+14,2 days 17:08:15.071347040,1.927800e+14,2 days 05:33:00,13800000000000,0 days 03:50:00
2,2024-LMV2YC,ผิดกฎจราจร,"สน.ทองหล่อ, เขตคลองเตย, สำนักงานตำรวจแห่งชาติ ...",จอดกึดขวางเหมือนเดิมทุกวัน ทุกเวลาครับ,"100.56348,13.73007",อาคาร Mille Malle ซอย สุขุมวิท 20 ถ. สุขุมวิท ...,คลองเตย,คลองเตย,กรุงเทพมหานคร,finish,...,1140000000000,27.762049,5.547520e+15,64 days 04:58:39.896619578,7.256029e+14,8 days 09:33:22.914968887,6.078600e+14,7 days 00:51:00,5520000000000,0 days 01:32:00
3,MPA9A3,เสียง,"ฝ่ายปกครอง เขตธนบุรี, เขตธนบุรี, กรุงเทพมหานคร...",ปัญหา: บริเวณวัดกัลยาณ์ พบช่วงเทศกาลจะมีการจุด...,"100.49133,13.74001",371 ซอย วัดกัลยาณ์ แขวงวัดกัลยา เขตธนบุรี กรุง...,วัดกัลยาณ์,ธนบุรี,กรุงเทพมหานคร,finish,...,1560000000000,28.075707,2.732807e+15,31 days 15:06:46.610110803,6.511666e+14,7 days 12:52:46.554736235,5.802300e+14,6 days 17:10:30,62760000000000,0 days 17:26:00
4,2024-H4HK7T,หาบเร่แผงลอย,"ฝ่ายเทศกิจ เขตจตุจักร, เขตจตุจักร, กรุงเทพมหานคร",บนสกายวอร์กหน้าห้างยูเนียนมอล์ ตั้งแผงกีดขวางค...,"100.56074,13.8144",30 ถนนพหลโยธิน แขวงจตุจักร เขตจตุจักร กรุงเทพม...,จตุจักร,จตุจักร,กรุงเทพมหานคร,finish,...,14400000000000,30.298249,8.940264e+14,10 days 08:20:26.423278997,2.489583e+14,2 days 21:09:18.329546155,2.487000e+14,2 days 21:05:00,94320000000000,1 days 02:12:00


In [73]:
# combined_exploded_averages_df['type'].value_counts().reset_index()
combined_exploded_averages_df_bkk['type'].value_counts().reset_index()

,type,count
0,อื่นๆ,202584
1,ผิดกฎจราจร,102147
2,ทางเท้า,100369
3,ความปลอดภัย,70976
4,ถนน,59482
...,...,...
65,ด้านบริการ -> ไฟดับ,1
66,อาคาร/อุปกรณ์ชำรุด,1
67,ห้องน้ำคนพิการ,1
68,Public Event ขออนุญาตจัดกิจกรรม,1


In [74]:
# combined_exploded_averages_df['type'].value_counts().reset_index().to_csv("combined_exploded_averages_df['type'].value_counts().csv", index=False)
# To see every case.

# combined_exploded_averages_df_bkk['type'].value_counts().reset_index().to_csv("combined_exploded_averages_df_bkk['type'].value_counts().csv", index=False)
# To see every case.

In [75]:
# combined_exploded_averages_df['province'].value_counts().reset_index()
combined_exploded_averages_df_bkk['province'].value_counts().reset_index()

,province,count
0,กรุงเทพมหานคร,896289


In statistics, sample size (n) should be larger than 30 to be 'sufficiently large' for central limit theoram to apply.

In [76]:
type_count = combined_exploded_averages_df_bkk['type'].value_counts().reset_index()

In [79]:
type_count[type_count['count'] >= 30]

,type,count
0,อื่นๆ,202584
1,ผิดกฎจราจร,102147
2,ทางเท้า,100369
3,ความปลอดภัย,70976
4,ถนน,59482
5,ความสะอาด,51171
6,ไฟฟ้า,44358
7,จุดเสี่ยง,42428
8,ต้นไม้,33463
9,น้ำท่วม,27470


In [80]:
combined_exploded_averages_df_bkk = pd.merge(left=combined_exploded_averages_df_bkk, right=type_count[type_count['count'] >= 30], how='left', on='type')

In [81]:
combined_exploded_averages_df_bkk['count'].isnull().sum()

np.int64(93)

In [82]:
combined_exploded_averages_df_bkk.dropna(subset=['count'], inplace=True)

In [84]:
combined_exploded_averages_df_bkk['count'].isnull().sum()

np.int64(0)

In [85]:
combined_exploded_averages_df_bkk.to_csv('Combined_Cases_And_Logs_Exploded_Averages_Bangkok_Plot.csv', index=False)

#### Plot duration for each case by type

In [86]:
combined_exploded_averages_df_bkk = pd.read_csv('Combined_Cases_And_Logs_Exploded_Averages_Bangkok_Plot.csv')

In [ ]:
type_list = combined_exploded_averages_df_bkk['type']

### Some other thing idk

# Other Stuff 2